In [1]:
import tensorflow as tf
import numpy as np

import tf_encrypted as tfe

/anaconda3/envs/py35_int64/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.models import Sequential

## Plaintext model

In [3]:
model_k = Sequential([Dense(5, batch_input_shape=[1, 5]), 
                      Dense(2)])

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
data = np.ones(shape=[1, 5]).astype('float')
model_k.predict(data)

array([[-0.23738731,  0.42355973]], dtype=float32)

## Create tfe keras model wigth from_config and set weights

In [5]:
keras_weights = model_k.get_weights()
config = model_k.get_config()

l1 = model_k.layers[0]
l2 = model_k.layers[1]

keras_weights_l1 = l1.get_weights()
keras_weights_l2 = l2.get_weights()

In [6]:
tf.reset_default_graph()

with tfe.protocol.SecureNN():
    
    tfe_sequential = tfe.keras.Sequential([])
    
    tfe_model = tfe_sequential.from_config(config)
    
    x = tfe.define_private_variable(data)

In [7]:
with tfe.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    tfe_model.set_weights(keras_weights)
    
    y = tfe_model(x)
    actual = sess.run(y.reveal())
    
    print(actual)

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2']


[[-0.23731139  0.42341107]]


## Create tfe keras model wigth model_from_config and set weights

In [8]:
tf.reset_default_graph()

with tfe.protocol.SecureNN():
    
    tfe_model = tfe.keras.models.model_from_config(config)
    
    x = tfe.define_private_variable(data)

In [9]:
with tfe.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    tfe_model.set_weights(keras_weights)

    y = tfe_model(x)
    actual = sess.run(y.reveal())
    
    print(actual)

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2']


[[-0.23731139  0.42356348]]


## Create tfe keras model wigth from_config and set weights for each layer individually

In [10]:
tf.reset_default_graph()

with tfe.protocol.SecureNN():
    
    tfe_sequential = tfe.keras.Sequential([])
    
    tfe_model = tfe_sequential.from_config(config)
    
    x = tfe.define_private_variable(data)

In [11]:
with tfe.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    l1 = tfe_model.layers[0]
    l2 = tfe_model.layers[1]
    
    l1.set_weights(keras_weights_l1)
    l2.set_weights(keras_weights_l2)
    
    y = tfe_model(x)
    actual = sess.run(y.reveal())
    
    print(actual)

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2']


[[-0.23731139  0.42356348]]


## Create tfe keras model with clone_model

In [12]:
with tfe.protocol.SecureNN():
    tfe_model = tfe.keras.models.clone_model(model_k)
    
    x = tfe.define_private_variable(data)

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2']


In [13]:
tfe_sess = tfe_model._tfe_session

In [14]:
with tfe_sess:
    # won't work if we re-initialize all the weights
    # with tfe_sess.run(tf.global_variables_initializer())
    tfe_sess.run(x.initializer)
    y = tfe_model(x)
    print(tfe_sess.run(y.reveal()))

[[-0.23731139  0.42356348]]
